Import relevant libraries

In [ ]:
import dask.dataframe as dd
import os
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

from time import time
from sklearn.model_selection import KFold
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

Set working directory

In [ ]:
os.chdir(r'C:\Users\user\Documents\Degrees\Data Science MSc\8. 7150CEM Data Science Project (50 credits)')
os.getcwd()

Make a list that will be the collumn names for the PPD and read the PPD datatsets in

In [ ]:
names = ['tui', 'price', 'DoT',"POSTCODE", 'propertyType', 'Age','Duration',
         'PAON', 'SAON', 'street', 'Locality', 'TownCity', 'District', 'Country',
         'ppdCat', 'RecStatus']

In [ ]:
#Use Pandas to read in the 2020 PPD
pp2020Data = pd.read_csv('price_paid_2020.csv',header=None, names=names)

#Use dask to read in the full 1995-2021 PPD
ppFullData = dd.read_csv('price_paid_complete.csv',header=None, names=names)

### ITERATION/SUBSET 1

In [ ]:
#Creat a Coventry subset of the 2020 PPD (SUBSET 1)
CoventryPP2020 = pp2020Data[pp2020Data.TownCity == 'COVENTRY']
len(CoventryPP2020) #4357
#To make a column with full addresses. 
CoventryPP2020['FullAddress'] = CoventryPP2020[['PAON', 'SAON', 'street']].astype(str).agg(','.join, axis=1)
CoventryPP2020['FullAddress'].replace({'nan,':' '}, regex=True, inplace = True)

#To make an EPC dataset for Coventry properties 
CoventryEPC2020 = pd.read_csv(r'all-domestic-certificates\domestic-E08000026-Coventry\certificates.csv')
len(CoventryEPC2020) #
#Upper case EPC
CoventryEPC2020['ADDRESS'] = CoventryEPC2020['ADDRESS'].str.upper()

#To merge the datasets
MergedData = CoventryPP2020.merge(CoventryEPC2020, how='inner', left_on=['FullAddress','POSTCODE'], right_on=['ADDRESS','POSTCODE'])
len(MergedData) #4030


### ITERATION/SUBSET 2

In [ ]:
#Creat a 2020 Subset with the largest cities (SUBSET 2)
ManchesterPP2020 = pp2020Data[pp2020Data.TownCity == 'MANCHESTER']
len(ManchesterPP2020) #13936
#Selecting Reading
NothamPP2020 = pp2020Data[pp2020Data.TownCity == 'NOTTINGHAM']
len(NothamPP2020) #10914
#Selecting Birmingham
BhamPP2020 = pp2020Data[pp2020Data.TownCity == 'BIRMINGHAM']
len(BhamPP2020) #11492
#Selecting Cardif
BristolPP2020 = pp2020Data[pp2020Data.TownCity == 'BRISTOL']
len(BristolPP2020) #11745
#Selecting Liverpool
LiverpoolPP2020 = pp2020Data[pp2020Data.TownCity == 'LIVERPOOL']
len(LiverpoolPP2020) #9776
#Selecting Leeds
LeedsPP2020 = pp2020Data[pp2020Data.TownCity == 'LEEDS']
len(LeedsPP2020) #9479
#Selecting Birmingham
SheffieldPP2020 = pp2020Data[pp2020Data.TownCity == 'SHEFFIELD']
len(SheffieldPP2020) #8183
#Selecting Leicester
LeicesterPP2020 = pp2020Data[pp2020Data.TownCity == 'LEICESTER']
len(LeicesterPP2020) #6927

#To merge the PP data for all 9 cities
PPframes = [CoventryPP2020, ManchesterPP2020, NothamPP2020, BhamPP2020, 
            BristolPP2020,LiverpoolPP2020, LeedsPP2020, SheffieldPP2020, 
            LeicesterPP2020]
PP_9_cities = pd.concat(PPframes)
len(PP_9_cities) #86809

#To make a column with full addresses. 
PP_9_cities['FullAddress'] = PP_9_cities[['PAON', 'SAON', 'street']].astype(str).agg(','.join, axis=1)
PP_9_cities['FullAddress'].replace({'nan,':' '}, regex=True, inplace = True)

##Making EPC dataframe
#To make an EPC dataset for Coventry properties 
CoventryEPC = pd.read_csv(r'all-domestic-certificates\domestic-E08000026-Coventry\certificates.csv')
len(CoventryEPC) #125,481

#To make an EPC dataset for Manchester properties 
ManchesterEPC = pd.read_csv(r'all-domestic-certificates\domestic-E08000003-Manchester\certificates.csv')
len(ManchesterEPC) #245,779

#To make an EPC dataset for Nottingham properties 
NothamEPC = pd.read_csv(r'all-domestic-certificates/domestic-E06000018-Nottingham\certificates.csv')
len(NothamEPC) #161837

#To make an EPC dataset for Birmingham properties 
BhamEPC = pd.read_csv(r'all-domestic-certificates/domestic-E08000025-Birmingham\certificates.csv')
len(BhamEPC) #393335

#To make an EPC dataset for Bristol properties 
BristolEPC = pd.read_csv(r'all-domestic-certificates/domestic-E06000023-Bristol-City-of\certificates.csv')
len(BristolEPC) #173272

#To make an EPC dataset for Liverpool properties 
LiverpoolEPC = pd.read_csv(r'all-domestic-certificates/domestic-E08000012-Liverpool\certificates.csv')
len(LiverpoolEPC) #201791

#To make an EPC dataset for Leeds properties 
LeedsEPC = pd.read_csv(r'all-domestic-certificates/domestic-E08000035-Leeds\certificates.csv')
len(LeedsEPC) #315050

#To make an EPC dataset for Sheffield properties 
SheffieldEPC = pd.read_csv(r'all-domestic-certificates/domestic-E08000019-Sheffield\certificates.csv')
len(SheffieldEPC) #192783

#To make an EPC dataset for Leicester properties 
LeicesterEPC = pd.read_csv(r'all-domestic-certificates/domestic-E06000016-Leicester\certificates.csv')
len(LeicesterEPC) #124361

#To merge the EPC data for all 9 cities
EPCframes = [CoventryEPC, ManchesterEPC, NothamEPC, BhamEPC, BristolEPC,
             LiverpoolEPC, LeedsEPC, SheffieldEPC, LeicesterEPC]
EPC_9_cities = pd.concat(EPCframes)
#To make the address upper case as the address in the PP data are uppercase too
EPC_9_cities['ADDRESS'] = EPC_9_cities['ADDRESS'].str.upper()
len(EPC_9_cities) #1933689

MergedData = PP_9_cities.merge(EPC_9_cities, how='inner', left_on=['FullAddress','POSTCODE'], right_on=['ADDRESS','POSTCODE'])
len(MergedData)
#45053

#Quick Read-in for subset 2
#NoDupData = pd.read_csv('cities_9_PP.csv')

### ITERATION/SUBSET 3

In [ ]:
#Creat a Coventry Subset with all the data from 1995-2021
#Create a Coventry subset
CoventryPP = ppFullData[ppFullData.TownCity == 'COVENTRY'].compute()
len(CoventryPP)

#####CREATE CSV FOR QUICK LOADING IN FUTURE
CoventryPP.to_csv('CoventryPricePaid.csv',index=False)

#Quick Read in of CoventryPP
CoventryPP = pd.read_csv('CoventryPricePaid.csv')
len(CoventryPP) #151761

#To make a column with full addresses to match that found in the EPC
CoventryPP['FullAddress'] = CoventryPP[['PAON', 'SAON', 'street']].astype(str).agg(','.join, axis=1)
CoventryPP['FullAddress'].replace({'nan,':' '}, regex=True, inplace = True)


#CoventryEPC
CoventryEPC = pd.read_csv(r'all-domestic-certificates\domestic-E08000026-Coventry\certificates.csv')
len(CoventryEPC) #125481
#Upper case the address to match the way it is in the PPD
CoventryEPC['ADDRESS'] = CoventryEPC['ADDRESS'].str.upper()

#Merge the Coventry PPD and EPC based on address and postcode
MergedData = CoventryPP.merge(CoventryEPC, how='inner', left_on=['FullAddress','POSTCODE'], right_on=['ADDRESS','POSTCODE'])
len(MergedData) #109997

### ALL SUBSETS USE THIS CODE TO REMOVE DUPLICATES

In [ ]:
#Removing Duplicates by keeping only the latest EPC Inspection
MergedData["INSPECTION_DATE"] = pd.to_datetime(MergedData["INSPECTION_DATE"] )
NoDupData = MergedData.sort_values(by="INSPECTION_DATE")
NoDupData.drop_duplicates(subset=['BUILDING_REFERENCE_NUMBER'], keep='last', inplace=True)
len(NoDupData) #45697


#Create year column (For subset 3 only)
NoDupData['Year'] = pd.DatetimeIndex(NoDupData['DoT']).year

#### USE FOR ITERATION 1 AND 2 ###
UsefulData = NoDupData[['price','DoT','POSTCODE', 'FullAddress', 'TownCity',
                        'BUILDING_REFERENCE_NUMBER','propertyType','Age','Duration',
                        'CURRENT_ENERGY_RATING', 'CURRENT_ENERGY_EFFICIENCY',
                        'PROPERTY_TYPE', 'BUILT_FORM', 'ENERGY_CONSUMPTION_CURRENT',
                        'LIGHTING_COST_CURRENT', 'HEATING_COST_CURRENT',
                        'HOT_WATER_COST_CURRENT','TOTAL_FLOOR_AREA', 'EXTENSION_COUNT',
                        'NUMBER_HABITABLE_ROOMS','CONSTRUCTION_AGE_BAND','FLOOR_HEIGHT']]



#### USE FOR ITERATION 3 ###
UsefulData = NoDupData[['price','DoT','POSTCODE', 'FullAddress', 'TownCity',
                        'BUILDING_REFERENCE_NUMBER','propertyType','Age','Duration',
                        'CURRENT_ENERGY_RATING', 'CURRENT_ENERGY_EFFICIENCY',
                        'PROPERTY_TYPE', 'BUILT_FORM', 'ENERGY_CONSUMPTION_CURRENT',
                        'LIGHTING_COST_CURRENT', 'HEATING_COST_CURRENT',
                        'HOT_WATER_COST_CURRENT','TOTAL_FLOOR_AREA', 'EXTENSION_COUNT',
                        'NUMBER_HABITABLE_ROOMS','CONSTRUCTION_AGE_BAND','FLOOR_HEIGHT',
                        'Year']]


### Exploratory Data Analysis & Missing Values

In [ ]:
### MISSING VALUES ###
UsefulData.info()
#Check missing values
UsefulData.isnull().sum()
#EXTENSION_COUNT               4356
#NUMBER_HABITABLE_ROOMS        4356
#CONSTRUCTION_AGE_BAND          191
UsefulData.isna().sum()
#Check the % of missing values
round(UsefulData.isnull().sum()*100/UsefulData.shape[0],2).sort_values(ascending=False).head(20)
#Drop the rows where there are missing values in number of habitable rooms
#These are the same properties where extension count and construction age band 
#information was missing
UsefulData.dropna(how='any',subset=['NUMBER_HABITABLE_ROOMS'], inplace = True)
UsefulData.dropna(how='any',subset=['CONSTRUCTION_AGE_BAND'], inplace = True)

#Most properties do not have information for floor height so this column can be dropped completely

UsefulData.Duration.value_counts()
#Relates to the tenure: F = Freehold, L= Leasehold etc.
#F    37409
#L     3734
#U        3
#Will remove the 3 U as these are considered missing values
UsefulData = UsefulData[UsefulData.Duration != 'U']
len(UsefulData) #41173

UsefulData.propertyType.value_counts()
#D = Detached, S = Semi-Detached, T = Terraced, F = Flats/Maisonettes, O = Other
#T    24061
#S    10340
#D     3346
#F     3331
#O       98


UsefulData.PROPERTY_TYPE.value_counts()
#House         35969
#Flat           2856
#Bungalow       1485
#Maisonette      866

UsefulData.BUILT_FORM.value_counts()
#Together with the PROPERTY TYPE from EPC, the Build Form produces a structured 
#description of the property. So will drop property type from price paid
#Mid-Terrace             17006
#Semi-Detached           11636
#End-Terrace              8134
#Detached                 3998
#Enclosed End-Terrace      208
#NO DATA!                  114
#Enclosed Mid-Terrace       80
#will remove the rows where there is no data if keeping the built_form variable
#UsefulData = UsefulData[UsefulData.BUILT_FORM != 'NO DATA!']

UsefulData.Age.value_counts()
#Y = a newly built property, N = an established residential building
#N    39553
#Y     1623

UsefulData.CONSTRUCTION_AGE_BAND.value_counts()
#Construction age band columns from the EPC dataset is much more nuanced than 
#the old/new (age) column in the price paid dataset. So will drop the Age from PP data

#England and Wales: 1930-1949       12568
#England and Wales: 1900-1929        9574
#England and Wales: 1950-1966        8541
#England and Wales: 1967-1975        3442
#England and Wales: before 1900      1371
#England and Wales: 1996-2002        1199
#England and Wales: 2003-2006        1152
#England and Wales: 1983-1990        1048
#England and Wales: 1976-1982        1016
#England and Wales: 2007 onwards      634
#England and Wales: 1991-1995         601
#INVALID!                              29
#England and Wales: 2007-2011           1
#will remove invalid rows as missing values as well as the row where there is only 1 oveservation
UsefulData = UsefulData[UsefulData.CONSTRUCTION_AGE_BAND != 'INVALID!']
UsefulData = UsefulData[UsefulData.CONSTRUCTION_AGE_BAND != 'England and Wales: 2007-2011']
len(UsefulData) #41143


UsefulData.NUMBER_HABITABLE_ROOMS.value_counts()
#includes kitchen and living room
#5.0     14524
#4.0     13199
#3.0      5569
#6.0      4098
#7.0      1667
#2.0       827
#8.0       677
#9.0       303
#10.0      132
#1.0        58
#11.0       42
#12.0       21
#14.0        8
#13.0        6
#15.0        4
#16.0        4
#20.0        2
#22.0        1
#17.0        1
#Exclude properties with >13 rooms as there are so few of them
UsefulData = UsefulData[UsefulData.NUMBER_HABITABLE_ROOMS < 13]
len(UsefulData)#41117

UsefulData.CURRENT_ENERGY_RATING.value_counts()
#D    22383
#E     8568
#C     8165
#F     1204
#B      472
#G      302
#A       23

UsefulData.EXTENSION_COUNT.value_counts()
#0.0    20290
#1.0    16346
#2.0     3884
#3.0      494
#4.0      103


### Removing Outliers for subset 1&2

In [ ]:
UsefulData.price.describe()

plt.figure()
sns.boxplot(data = UsefulData, y = 'price')
plt.xlabel('All properties', fontsize = 10)
plt.ylabel('Price', fontsize = 10)
plt.title('Boxplot for Price Before Removing Outliers')
plt.show()

Q1=UsefulData['price'].quantile(0.25)
Q3=UsefulData['price'].quantile(0.75)
IQR=Q3-Q1
print(Q1)
print(Q3)
print(IQR)
Lower_Whisker = Q1-1.5*IQR
Upper_Whisker = Q3+1.5*IQR
print(Lower_Whisker, Upper_Whisker)

UsefulData = UsefulData[UsefulData['price']< Upper_Whisker]
UsefulData = UsefulData[UsefulData['price']> Lower_Whisker]
sns.boxplot(data = UsefulData, y = 'price')
UsefulData.price.describe()
len(UsefulData) #28352


sns.distplot(UsefulData['price'])
sns.boxplot(data = UsefulData, y = 'price')
UsefulData.price.describe()

min(UsefulData.price)

UsefulData.price
UsefulData.shape # There are now 6156 rows (was 6469 before so lost 313 rows)

#2 OUTLIERS IN FLOOR AREA
UsefulData.TOTAL_FLOOR_AREA.describe()
sns.boxplot(data = UsefulData, y = 'TOTAL_FLOOR_AREA')
sns.distplot(UsefulData['TOTAL_FLOOR_AREA'])

UsefulData = UsefulData[UsefulData.TOTAL_FLOOR_AREA < 150]
UsefulData = UsefulData[UsefulData['TOTAL_FLOOR_AREA'] >30]

#Looking for the upper & lower whisker as well as the interquartile range
Q1=UsefulData['TOTAL_FLOOR_AREA'].quantile(0.25)
Q3=UsefulData['TOTAL_FLOOR_AREA'].quantile(0.75)
IQR=Q3-Q1
print(Q1)
print(Q3)
print(IQR)
Lower_Whisker = Q1-1.5*IQR
Upper_Whisker = Q3+1.5*IQR
print(Lower_Whisker, Upper_Whisker)

UsefulData = UsefulData[UsefulData['TOTAL_FLOOR_AREA']< Upper_Whisker]
UsefulData = UsefulData[UsefulData['TOTAL_FLOOR_AREA']> Lower_Whisker]

sns.distplot(UsefulData['TOTAL_FLOOR_AREA'])
sns.boxplot(data = UsefulData, y = 'TOTAL_FLOOR_AREA')
UsefulData.TOTAL_FLOOR_AREA.describe()
UsefulData.shape 

#2454 for subset 1
#27341 for 9 subset 2

### Removing Outliers for subset 3

In [ ]:
UsefulData.price.describe()

#Reveals that max price is 3.174837e+06
#min price is 7.250000e+02

plt.figure()
sns.boxplot(data = UsefulData, y = 'price')
plt.xlabel('All properties', fontsize = 10)
plt.ylabel('Price', fontsize = 10)
plt.title('Boxplot for Price Before Removing Outliers')
plt.show()

#Get rid of extremes (properties over £600,000 and less than 10,000)
NoOutlieData = UsefulData[UsefulData.price < 1500000]

#Now look at boxplot
plt.figure()
plt.xlabel('Price', fontsize = 10)
plt.ylabel('Density', fontsize = 10)
plt.title('Distribution of Price')
sns.distplot(NoOutlieData['price'])
plt.show()

len(NoOutlieData) #40913

#Boxplot to show price by year
plt.figure()
plt.xlabel('Year', fontsize = 10)
plt.ylabel('Price', fontsize = 10)
#plt.title('Price Each Year Before Removing Outliers')
ax = sns.boxplot(data = NoOutlieData, x='Year',y='price')
for item in ax.get_xticklabels():
    item.set_rotation(90)
plt.show()

#Displot to show that price is not normally distributed and therefore the 
#interquartile range method should be used to remove outliers.
sns.distplot(NoOutlieData['price'])

#Create functions to address outliers in price
def OutlierForYear(NoOutlieData, year):
    """Function to remove the rows with outliers in price
    for a given year. Outliers removed using the interquartile
    range method."""
    frame = NoOutlieData[NoOutlieData.Year == year]
    Q1=frame['price'].quantile(0.25)
    Q3=frame['price'].quantile(0.75)
    IQR=Q3-Q1
    #print(Q1)
    #print(Q3)
    #print(IQR)
    Lower_Whisker = Q1-1.5*IQR
    Upper_Whisker = Q3+1.5*IQR
    #print(Lower_Whisker, Upper_Whisker)
    frame = frame[frame['price']< Upper_Whisker]
    frame = frame[frame['price']> Lower_Whisker]
    #sns.boxplot(data = frame, y = 'price')
    frame.price.describe()
    #print(len(frame)) #39192
    return(frame)

def All_Outliers():
    """Function to apply the outlier removal function to each year in the 
    PPD & EPC merged dataset."""
    OutliersGone=pd.DataFrame()
    for year in NoOutlieData.Year.unique():
        frame = OutlierForYear(NoOutlieData, year)
        OutliersGone=OutliersGone.append(frame,ignore_index=True)
    return(OutliersGone)

#Apply the function    
Outlier_Free = All_Outliers()
len(Outlier_Free) #38452

#Boxplot to show price by year after removing outliers
ax = sns.boxplot(data = Outlier_Free, x='Year',y='price')
for item in ax.get_xticklabels():
    item.set_rotation(90)
    
Outlier_Free.price.describe()

#Now address outliers in total floor area
Outlier_Free.TOTAL_FLOOR_AREA.describe()
#Boxplot in total floor are BEFORE removing outliers
sns.boxplot(data = Outlier_Free, y = 'TOTAL_FLOOR_AREA')
#Displot shows it is not evenly distributed either.
sns.distplot(Outlier_Free['TOTAL_FLOOR_AREA'])

#Use the interquartile range method to remove outliers.  
Q1=Outlier_Free['TOTAL_FLOOR_AREA'].quantile(0.25)
Q3=Outlier_Free['TOTAL_FLOOR_AREA'].quantile(0.75)
IQR=Q3-Q1
print(Q1)
print(Q3)
print(IQR)
Lower_Whisker = Q1-1.5*IQR
Upper_Whisker = Q3+1.5*IQR
print(Lower_Whisker, Upper_Whisker)

Outlier_Free = Outlier_Free[Outlier_Free['TOTAL_FLOOR_AREA']< Upper_Whisker]
Outlier_Free = Outlier_Free[Outlier_Free['TOTAL_FLOOR_AREA']> Lower_Whisker]

#Boxplot for total floor area after removing outliers
sns.boxplot(data = Outlier_Free, y = 'TOTAL_FLOOR_AREA')
Outlier_Free.TOTAL_FLOOR_AREA.describe()
Outlier_Free.shape # 37086

### Further Exploratory Data Analysis of Categorical Features After Removing Outliers

In [ ]:
#For property type
Outlier_Free['propertyType']=Outlier_Free['propertyType'].map({'F': 'Flat/Maissonette', 
                                                               'T': 'Terraced',
                                                               'O': 'Other',
                                                               'D': 'Detached',
                                                               'S': 'Semi-detached'})
#D = Detached, S = Semi-Detached, T = Terraced, F = Flats/Maisonettes, O = Other

plt.figure()
ax = sns.boxplot(data = Outlier_Free, x='propertyType',y='price')
for item in ax.get_xticklabels():
    item.set_rotation(90)
plt.show()

#For energy rating
plt.figure()
ax = sns.boxplot(data = Outlier_Free, x='CURRENT_ENERGY_RATING',y='price')
plt.show()

#For duration
Outlier_Free['Duration']=Outlier_Free['Duration'].map({'F': 'Freehold', 'L': 'Leasehold'})
plt.figure()
ax = sns.boxplot(data = Outlier_Free, x='Duration',y='price')
plt.show()

#For construction age
Outlier_Free['CONSTRUCTION_AGE_BAND']=Outlier_Free['CONSTRUCTION_AGE_BAND'].map({ 
                                                               'England and Wales: 1930-1949': '1930-1949',
                                                               'England and Wales: 1900-1929': '1900-1929',
                                                               'England and Wales: 1950-1966': '1950-1966',
                                                               'England and Wales: 1967-1975': '1967-1975',
                                                               'England and Wales: before 1900': 'Before 1900',
                                                               'England and Wales: 2003-2006': '2003-2006',
                                                               'England and Wales: 1996-2002': '1996-2002',
                                                               'England and Wales: 1976-1982': '1976-1982',
                                                               'England and Wales: 1983-1990': '1983-1990',
                                                               'England and Wales: 2007 onwards': '2007 onwards',
                                                               'England and Wales: 1991-1995': '1991-1995'})
plt.figure()
ax = sns.boxplot(data = Outlier_Free, x='CONSTRUCTION_AGE_BAND',y='price')
for item in ax.get_xticklabels():
    item.set_rotation(90)
plt.show()

### Select only the variables to be used in ML algorithm & Exploratory Data Analysis of Numerical Features

In [ ]:
### USE FOR ITERATION 1 AND 2 ###
Data = UsefulData[['price','Duration',
                        'CURRENT_ENERGY_RATING', 
                        'propertyType', 
                        'TOTAL_FLOOR_AREA', 
                        'NUMBER_HABITABLE_ROOMS',
                        'EXTENSION_COUNT',
                        'CONSTRUCTION_AGE_BAND']]

### USE FOR ITERATION 3 ###
Data = Outlier_Free[['price','Duration',
                        'CURRENT_ENERGY_RATING', 
                        'propertyType', 
                        'TOTAL_FLOOR_AREA', 
                        'NUMBER_HABITABLE_ROOMS',
                        'EXTENSION_COUNT',
                        'CONSTRUCTION_AGE_BAND',
                        'Year']]

### Use to create a heatmap of the numerical features
For_Heatmap = Outlier_Free[['price',
                        'TOTAL_FLOOR_AREA', 
                        'NUMBER_HABITABLE_ROOMS',
                        'EXTENSION_COUNT',
                        'Year']]

#To plot Heatmap
mask = np.triu(np.ones_like(For_Heatmap.corr(), dtype=np.bool))
plt.figure()
heatmap = sns.heatmap(For_Heatmap.corr(),mask=mask, vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);
plt.show()
#To see just correlations for price
plt.figure(figsize=(8, 12))
sns.set(font_scale=2)
heatmap = sns.heatmap(For_Heatmap.corr()[['price']].sort_values(by='price', ascending=False), vmin=-1, vmax=1, annot=True,  annot_kws={"size":30})
heatmap.set_title('Correlations Between Numerical Features & Sales Price', fontdict={'fontsize':20}, pad=16);



### Data Pre-processing

In [ ]:
#PREPROCESSING: split dataset into features (X) and labels (y)
X = Data.drop(columns=['price']) # the attributes
y = Data.price #the label
X.shape #(39491, 8)
y.shape

#PREPROCESSING: Covert ordinal categorical (the energy rating) to numerial
X['CURRENT_ENERGY_RATING']=X['CURRENT_ENERGY_RATING'].map({'A': 6, 'B': 5,'C': 4, 'D': 3,'E': 2, 'F': 1,'G': 0})

#PREPROCESSING: convert the rest of the categorical data to numerical forms. 
X = pd.get_dummies(X, drop_first= True)
X.shape #(39491, 20)

#(2454, 19) for iteration 1

#Scaling using MinMax 
cols = X.columns
X[cols] = MinMaxScaler().fit_transform(X[cols].values)
X 

#PREPROCESSING: Train-test splitting
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)
X_train.shape #(29618, 20)
X_test.shape #(9873, 20)


### Model Implementation

#### Feature Selection Function

In [ ]:
def fwd_ft_sel(model, feature_train, feature_test, label_train, seed):
    """Sequential Forward selection function"""
    #Set 10-fold strafified cross validation
    cv = KFold(n_splits=10, random_state = seed, shuffle = True) 
    #Set up sequential feature selector
    sfs1 = SFS(estimator = model,
            k_features = (1,20),
            forward = True,
            floating = False,
            verbose = 2,
            scoring = 'r2',
            cv=cv,
            n_jobs = -1)
    #Fit the model and time it
    tic_fwd = time()
    sfs1 = sfs1.fit(feature_train,label_train)
    toc_fwd = time()
    #Return important information
    print('\n=======Time Taken=======')
    time_taken = toc_fwd - tic_fwd
    print(time_taken)
    print('\n=======Best score=======')
    best_score = (sfs1.k_score_ )
    print(best_score)
    print('\n=======Best Features=======')
    best_features = sfs1.k_feature_names_ #features included
    print(best_features)
    print('\n=======No. of Features=======')
    no_features = len(best_features)
    print(no_features)
    #Plot features vs performance
    plt.figure()
    plot_sfs(sfs1.get_metric_dict(), kind='std_dev')
    plt.title('Sequential Forward Selection (w. StdDev)')
    plt.grid()
    plt.show()
    #Make new subset based on selected features
    feature_train_sfs = sfs1.transform(feature_train)
    feature_test_sfs = sfs1.transform(feature_test)
    #Return useful outputs
    return(time_taken, best_score, best_features,feature_train_sfs,feature_test_sfs)



#### Hyperparameter Optimisation Function

In [ ]:
def hyperparam_optim(model, param_grid, feature_train_sfs, label_train):
    """Hyperparameter Optimisation Function"""
    #set up grid search with parameter grid to be defined for each model
    grid = GridSearchCV(model, param_grid, 
                        cv=10, 
                        scoring='r2', 
                        verbose = True, 
                        n_jobs = -1)
    #Fit the model on train set transformed by slected features and time it
    tic_fwd = time()
    grid.fit(feature_train_sfs,label_train)
    toc_fwd = time()
    print('\n=======Time Taken=======')
    time_taken = toc_fwd - tic_fwd
    print(time_taken)
    print('\n=======Best score=======')
    best_score = (grid.best_score_)
    print(best_score)
    print('\n=======Best Parameters=======')
    best_parameters = grid.best_params_ #features included
    print(best_parameters)
    #Return useful outputs
    return(time_taken, best_score, best_parameters)

#### Function to get metrics and graphs

In [ ]:
def get_metrics(model, feature_train_sfs, feature_test_sfs, label_train, label_test):
    """Function to fit model and return metrics & plots"""
    model.fit(feature_train_sfs,label_train)
    train_y_pred = model.predict(feature_train_sfs)
    test_y_pred = model.predict(feature_test_sfs)
    
    train_y_pred = model.predict(feature_train_sfs)
    test_y_pred = model.predict(feature_test_sfs)
    
    train_r2_score = metrics.r2_score(label_train, train_y_pred)
    test_r2_score = metrics.r2_score(label_test, test_y_pred)
    
    train_mae_score = metrics.mean_absolute_error(label_train, train_y_pred)
    test_mae_score = metrics.mean_absolute_error(label_test, test_y_pred)
    
    train_rmse_score = metrics.mean_squared_error(label_train, train_y_pred)
    test_rmse_score = metrics.mean_squared_error(label_test, test_y_pred)
    
    residuals = (label_test - test_y_pred)

    # Visualizing Our Training predictions
    plt.figure()
    plt.xlabel('Actual price')
    plt.ylabel('Predicted Price') 
    plt.scatter(label_train,train_y_pred, s=1, c='black')
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.title('Actual vs Predicred Price for Training Data')
    # Perfect predictions
    plt.plot(y_train,y_train,'r')
    plt.show()

    # Visualizing Our Test predictions
    plt.figure()
    plt.xlabel('Actual price')
    plt.ylabel('Predicted Price') 
    plt.scatter(label_test,test_y_pred, s=1, c='black')
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.title('Actual vs Predicred Price for Test Data')
    # Perfect predictions
    plt.plot(y_test,y_test,'r')
    plt.show()

    # Visualizing Our Test residuals
    plt.figure()
    plt.xlabel('Predicted price')
    plt.ylabel('Residuals')
    plt.scatter(label_test,residuals, s=1, c='black')
    plt.title('Residuals Plot for Test Data')
    # Perfect predictions
    plt.axhline(y=0.5, color='r', linestyle='-')
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.show()
    
    return(train_r2_score, test_r2_score,
           train_mae_score, test_mae_score,
           train_rmse_score, test_rmse_score, residuals)

#### GBM Results

In [ ]:
seed=42
xgb_model = GradientBoostingRegressor(random_state = seed)
#Feature selection
xgb_fs_time_taken, xgb_fs_best_score, xgb_fs_best_features, xgb_X_train_sfs, xgb_X_test_sfs = fwd_ft_sel(xgb_model, X_train, 
                                                                                                         X_test, y_train, seed)

#Hyperparameter Optimisation
loss = ['squared_error', 'absolute_error']
learning_rate_options = [0.1,0.2,0.3]
xgb_param_grid = dict(loss = loss, 
                  learning_rate = learning_rate_options) 
xgb_ho_time_taken, xgb_ho_best_score, xgb_ho_best_parameters = hyperparam_optim(xgb_model, xgb_param_grid, xgb_X_train_sfs, y_train)

#Model Metrics on train & test set
xgb_op_model = GradientBoostingRegressor(random_state = seed, learning_rate = 0.3, loss = 'squared_error')
xgb_train_r2_score, xgb_test_r2_score, xgb_train_mae_score, xgb_test_mae_score, xgb_train_rmse_score, xgb_test_rmse_score, xgb_residuals = get_metrics(xgb_op_model, xgb_X_train_sfs, xgb_X_test_sfs, y_train, y_test)

#(0.7760370356050019,
# 0.7578738359394868,
# 22587.409672981692,
# 23033.567369496333,
# 986363596.2857233,
# 1060213841.3433646)

#### AdaBoost Results 

In [ ]:
seed=42
adb_model = AdaBoostRegressor(random_state = seed)
#Feature selection
adb_fs_time_taken, adb_fs_best_score, adb_fs_best_features, adb_X_train_sfs, adb_X_test_sfs = fwd_ft_sel(adb_model, X_train, 
                                                                                                         X_test, y_train, seed)
#Hyperparameter Optimisation
loss = ['linear', 'square', 'exponential']
n_estimators_options = [10, 50, 100]
adb_param_grid = dict(loss = loss, 
                  n_estimators = n_estimators_options) 
adb_ho_time_taken, adb_ho_best_score, adb_ho_best_parameters = hyperparam_optim(adb_model, adb_param_grid, adb_X_train_sfs, y_train)

#Model Metrics on train & test set
adb_op_model = AdaBoostRegressor(random_state = seed, loss = 'exponential', n_estimators = 10)
adb_train_r2_score, adb_test_r2_score, adb_train_mae_score, adb_test_mae_score, adb_train_rmse_score, adb_test_rmse_score, adb_residuals = get_metrics(adb_op_model, adb_X_train_sfs, adb_X_test_sfs, y_train, y_test)

#(0.6669311355459162,
# 0.6648810301488107,
# 28949.164508455662,
# 28818.761585453984,
# 1466880936.5030425,
# 1467407587.7407577)

#### RF Results

In [ ]:
seed=42
rf_model = RandomForestRegressor(random_state = seed)
#Feature selection
rf_fs_time_taken, rf_fs_best_score, rf_fs_best_features, rf_X_train_sfs, rf_X_test_sfs = fwd_ft_sel(rf_model, X_train, X_test, y_train, seed)
#Hyperparameter Optimisation
n_estimators_options = [10, 50, 100]
criterion_options = ['squared_error','absolute_error']
rf_param_grid = dict(n_estimators = n_estimators_options, 
                  criterion = criterion_options) 
rf_ho_time_taken, rf_ho_best_score, rf_ho_best_parameters = hyperparam_optim(rf_model, rf_param_grid, rf_X_train_sfs, y_train)
#Model Metrics on test set
rf_op_model = RandomForestRegressor(random_state = seed, criterion = 'absolute_error', n_estimators = 100)
rf_train_r2_score, rf_test_r2_score, rf_train_mae_score, rf_test_mae_score, rf_train_rmse_score, rf_test_rmse_score, rf_residuals = get_metrics(rf_op_model, rf_X_train_sfs, rf_X_test_sfs, y_train, y_test)

#(0.9508126267114178,
# 0.7217966563391365,
# 10259.624264686023,
# 24468.791757139967,
# 217926613.56862903,
# 1195910695.454584)

#### Graphs to present results for Subset 3

In [ ]:
#Fit times
adb_fit_time = adb_fs_time_taken + adb_ho_time_taken
rf_fit_time = rf_fs_time_taken + rf_ho_time_taken
xgb_fit_time = xgb_fs_time_taken + xgb_ho_time_taken

# simple plot for fit time
Fit_Times =  [adb_fit_time,rf_fit_time,xgb_fit_time]
index = ['ADB','RF','XGB']
Fit_Times_Results = pd.DataFrame({'Fit Time': Fit_Times}, index=index)

plt.figure()
Fit_Times_Results.plot.bar(rot=0, fontsize = 20, legend = None)
plt.title('Fit Time For Each Model', fontsize = 20)
plt.xlabel('Model', fontsize = 20)
plt.ylabel('Fit Time (Seconds)', fontsize = 20)
#plt.ylim(0.5,1.0)
plt.show()

###Stacked graph for computation times
FS_times = [adb_fs_time_taken, rf_fs_time_taken, xgb_fs_time_taken]
HypOp_times = [adb_ho_time_taken, rf_ho_time_taken, xgb_ho_time_taken]
labels = index


fig, ax = plt.subplots()

ax.bar(labels, FS_times, label='Feature Selection')
ax.bar(labels, HypOp_times, bottom=FS_times,
       label='Model Specification')

ax.set_ylabel('Computaton Time (Seconds)')
ax.set_xlabel('Model')
#ax.set_title('')
ax.legend()

plt.show()

#Plots of metric measures
#r2
r2_train =  [adb_train_r2_score,rf_train_r2_score,xgb_train_r2_score]
r2_test =  [adb_test_r2_score,rf_test_r2_score,xgb_test_r2_score]
index = ['ADB','RF','XGB']
R2_Results = pd.DataFrame({'Train': r2_train,
                           'Test' : r2_test }, index=index)

#mae
mae_train =  [adb_train_mae_score,rf_train_mae_score,xgb_train_mae_score]
mae_test =  [adb_test_mae_score,rf_test_mae_score,xgb_test_mae_score]
index = ['ADB','RF','XGB']
MAE_Results = pd.DataFrame({'Train': mae_train,
                            'Test': mae_test}, index=index)
plt.figure()
MAE_Results.plot.bar(rot=0, fontsize = 20)
plt.title('MAE scores: Train vs Test Set', fontsize = 20)
plt.xlabel('Model', fontsize = 20)
plt.ylabel('MAE', fontsize = 20)
plt.ylim(5000,32500)
plt.show()

#rmse
rmse_train =  [adb_train_rmse_score,rf_train_rmse_score,xgb_train_rmse_score]
rmse_test =  [adb_test_rmse_score,rf_test_rmse_score,xgb_test_rmse_score]
index = ['ADB','RF','XGB']
rmse_Results = pd.DataFrame({'Train': rmse_train,
                             'Test': rmse_test}, index=index)
plt.figure()
rmse_Results.plot.bar(rot=0, fontsize = 20)
plt.title('RMSE scores:Train vs Test Set', fontsize = 20)
plt.xlabel('Model', fontsize = 20)
plt.ylabel('RMSE', fontsize = 20)
#plt.ylim(0.8,1.0)
plt.show()

#### Graphs to compare Subset 1,2 & 3 Results

In [ ]:
#### Comparing subset 1,2 & 3
#Iteration 1
Subset_1_train_r2 = 0.8208950689636683
Subset_1_test_r2 = 0.32077186281664816

#Iteration 2
Subset_2_train_r2 = 0.4686699432202833
Subset_2_test_r2 = 0.35621538902879553


Subset_3_train_r2 = 0.776
Subset_3_test_r2 = 0.758

#Plots of r2 for GBM for each subset
#r2
subsets_r2_train =  [Subset_1_train_r2, Subset_2_train_r2 ,Subset_3_train_r2]
subsets_r2_test =  [Subset_1_test_r2,Subset_2_test_r2,Subset_3_test_r2]
index = ['1','2','3']
Subset_R2_Results = pd.DataFrame({'Train': subsets_r2_train,
                           'Test' : subsets_r2_test }, index=index)

plt.figure()
Subset_R2_Results.plot.bar(rot=0, fontsize = 15, colormap='RdGy')
#plt.title('', fontsize = 20)
plt.xlabel('Data Subset', fontsize = 15)
plt.ylabel('R2 Score', fontsize = 15)
#plt.ylim(0.5,1.0)
plt.show()